In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [45]:
data_path = 'data/'

train_csv_file_path = data_path + 'train_ship_segmentations_v2.csv'
# train_csv_file_path = data_path + 'demo.csv'
train_image_path = data_path + 'train_v2/'

In [46]:
def load_csv(path):
    return tf.data.experimental.make_csv_dataset(
        path,
        batch_size=1, # required
        column_names=['ImageId', 'EncodedPixels'],
        num_epochs=1,
    )

train_csv = load_csv(train_csv_file_path)

# for batch in train_csv.take(1):
#     print(batch)

In [47]:
IMG_HEIGHT = 768
IMG_WIDTH = 768

def process_img(file):
    img = tf.io.read_file(train_image_path + file)
    img = tf.io.decode_jpeg(img, channels=3)
    return img

def decode_label_mask(encoded_pixels, image_height, image_width):
    mask = tf.zeros(image_height * image_width, dtype=tf.uint8)

    # Convert string to integer tensor
    pairs = tf.strings.to_number(tf.strings.split(encoded_pixels), out_type=tf.int32)

    # Iterate over pairs and update mask
    for i in range(0, len(pairs), 2):
        start = pairs[i] - 1
        run_length = pairs[i + 1]

        indices = tf.range(start, start + run_length)
        updates = tf.ones(run_length, dtype=tf.uint8)
        mask = tf.tensor_scatter_nd_update(mask, indices=tf.expand_dims(indices, axis=1), updates=updates)

    return  tf.reshape(mask, (image_height, image_width))

def process_label(label):
    return decode_label_mask(label, IMG_HEIGHT, IMG_WIDTH)

def process_batch(csv_item):
    X = process_img(csv_item['ImageId'])
    y = process_label(csv_item['EncodedPixels'])
    return X, y

train_ds = train_csv.unbatch().map(process_batch)
# train_ds.filter(lambda x, y: x.shape > 0)

L = train_ds.reduce(0, lambda x, _: x + 1)
print('L=', L)

for item, label in train_ds.take(1):
    background = item
    overlay = label

    fig, ax = plt.subplots()
    plt.imshow(background)
    
    ax.imshow(overlay, alpha=0.3)

2024-02-08 20:23:39.118245: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at scatter_nd_op.cc:218 : INVALID_ARGUMENT: indices[51] = [589824] does not index into shape [589824]


InvalidArgumentError: {{function_node __wrapped__ReduceDataset_Targuments_0_Tstate_1_output_types_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} indices[51] = [589824] does not index into shape [589824]
	 [[{{function_node while_body_99129}}{{node while/TensorScatterUpdate}}]] [Op:ReduceDataset] name: 